## MXNet

MXNet is a deep learning framework designed for both efficiency and flexibility. It allows you to mix the flavours of symbolic programming and imperative programming to maximize efficiency and productivity. In its core, a dynamic dependency scheduler that automatically parallelizes both symbolic and imperative operations on the fly. A graph optimization layer on top of that makes symbolic execution fast and memory efficient. The library is portable and lightweight, and it scales to multiple GPUs and multiple machines.

#### Simple multi-layer perceptron in MNIST

In [10]:
using MXNet

In [11]:
data = mx.Variable(:data)

MXNet.mx.SymbolicNode(MXNet.mx.MX_SymbolHandle(Ptr{Void} @0x000000000a2ea300))

In [12]:
fc1  = mx.FullyConnected(data = data, name=:fc1, num_hidden=128)
act1 = mx.Activation(data = fc1, name=:relu1, act_type=:relu)
fc2  = mx.FullyConnected(data = act1, name=:fc2, num_hidden=64)
act2 = mx.Activation(data = fc2, name=:relu2, act_type=:relu)
fc3  = mx.FullyConnected(data = act2, name=:fc3, num_hidden=10)

MXNet.mx.SymbolicNode(MXNet.mx.MX_SymbolHandle(Ptr{Void} @0x000000000a2eec40))

In [13]:
mlp  = mx.SoftmaxOutput(data = fc3, name=:softmax)

MXNet.mx.SymbolicNode(MXNet.mx.MX_SymbolHandle(Ptr{Void} @0x000000000a18efe0))

In [14]:
mlp = @mx.chain mx.Variable(:data)             =>
  mx.FullyConnected(name=:fc1, num_hidden=128) =>
  mx.Activation(name=:relu1, act_type=:relu)   =>
  mx.FullyConnected(name=:fc2, num_hidden=64)  =>
  mx.Activation(name=:relu2, act_type=:relu)   =>
  mx.FullyConnected(name=:fc3, num_hidden=10)  =>
  mx.SoftmaxOutput(name=:softmax)

MXNet.mx.SymbolicNode(MXNet.mx.MX_SymbolHandle(Ptr{Void} @0x000000000a29e6f0))

In [15]:
batch_size = 100
include(Pkg.dir("MXNet", "examples", "mnist", "mnist-data.jl"))
train_provider, eval_provider = get_mnist_providers(batch_size)

(MXNet.mx.MXDataProvider(MXNet.mx.MX_DataIterHandle(Ptr{Void} @0x000000000a3e15c0),Tuple{Symbol,Tuple}[(:data,(784,100))],Tuple{Symbol,Tuple}[(:softmax_label,(100,))],100,true,true),MXNet.mx.MXDataProvider(MXNet.mx.MX_DataIterHandle(Ptr{Void} @0x000000000404d180),Tuple{Symbol,Tuple}[(:data,(784,100))],Tuple{Symbol,Tuple}[(:softmax_label,(100,))],100,true,true))

In [16]:
model = mx.FeedForward(mlp, context=mx.gpu())

MXNet.mx.FeedForward(MXNet.mx.SymbolicNode(MXNet.mx.MX_SymbolHandle(Ptr{Void} @0x000000000a29e6f0)),[GPU0],#undef,#undef,#undef)

In [17]:
optimizer = mx.SGD(lr=0.1, momentum=0.9, weight_decay=0.00001)

MXNet.mx.SGD(MXNet.mx.SGDOptions(0.1,0.9,0,1.0e-5,MXNet.mx.LearningRate.Fixed(0.1),MXNet.mx.Momentum.Fixed(0.9)),#undef)

In [18]:
@time mx.fit(model, optimizer, train_provider, n_epoch=20, eval_data=eval_provider)


INFO: Start training on [GPU0]
INFO: Initializing parameters...
INFO: Creating KVStore...
INFO: TempSpace: Total 0 MB allocated on GPU0
INFO: Start training...
INFO: == Epoch 001 ==========
INFO: ## Training summary
INFO:           accuracy = 0.7612
INFO:               time = 1.2484 seconds
INFO: ## Validation summary
INFO:           accuracy = 0.9517
INFO: == Epoch 002 ==========
INFO: ## Training summary
INFO:           accuracy = 0.9570
INFO:               time = 1.4973 seconds
INFO: ## Validation summary
INFO:           accuracy = 0.9632
INFO: == Epoch 003 ==========
INFO: ## Training summary
INFO:           accuracy = 0.9721
INFO:               time = 1.4886 seconds
INFO: ## Validation summary
INFO:           accuracy = 0.9671
INFO: == Epoch 004 ==========
INFO: ## Training summary
INFO:           accuracy = 0.9788
INFO:               time = 1.4797 seconds
INFO: ## Validation summary
INFO:           accuracy = 0.9710
INFO: == Epoch 005 ==========
INFO: ## Training summary
INFO:   

 30.183343 seconds (25.61 M allocations: 785.703 MB, 3.29% gc time)


In [12]:
probs = mx.predict(model, eval_provider)

10x10000 Array{Float32,2}:
 5.79763e-13  8.34022e-22  1.0123e-11   …  2.20957e-17  1.18963e-17
 1.54726e-8   1.46249e-11  0.999997        7.51705e-23  2.22471e-24
 3.26623e-11  1.0          2.08448e-8      7.72174e-25  2.41204e-23
 3.57994e-10  1.76444e-18  2.36883e-11     3.47416e-17  1.87433e-28
 3.50414e-10  1.37008e-22  2.97204e-7      6.7937e-31   1.13583e-18
 4.16315e-12  3.84812e-19  4.02902e-9   …  1.0          2.60441e-19
 5.68832e-15  3.49655e-19  3.95294e-9      5.0934e-15   1.0        
 1.0          1.36276e-13  5.80916e-7      2.25628e-20  9.75627e-33
 2.91113e-13  6.68736e-22  2.39233e-6      3.49587e-13  3.3515e-20 
 4.67181e-7   6.65774e-28  3.33547e-9      7.99229e-20  1.18165e-19

In [14]:
# collect all labels from eval data
labels = Array[]
for batch in eval_provider
    push!(labels, copy(mx.get(eval_provider, batch, :softmax_label)))
end
labels = cat(1, labels...)


10000-element Array{Float32,1}:
 7.0
 2.0
 1.0
 0.0
 4.0
 1.0
 4.0
 9.0
 5.0
 9.0
 0.0
 6.0
 9.0
 ⋮  
 5.0
 6.0
 7.0
 8.0
 9.0
 0.0
 1.0
 2.0
 3.0
 4.0
 5.0
 6.0

In [15]:
# Now we use compute the accuracy
correct = 0
for i = 1:length(labels)
    # labels are 0...9
    if indmax(probs[:,i]) == labels[i]+1
        correct += 1
    end
end
accuracy = 100correct/length(labels)
println(mx.format("Accuracy on eval set: {1:.2f}%", accuracy))

Accuracy on eval set: 97.60%
